In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc, f1_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import os
seed = 45
from xgboost import XGBClassifier
#seed =145
% matplotlib inline

In [2]:
PATH='F:/AV/WNS'
OOF_PATH = 'F:/AV/WNS/oof'
submission_path = 'F:/AV/WNS/submission'
stack_submission_path = 'F:/AV/WNS/submission/stack'
train_csv = 'train_catboost.csv'
submit_csv = 'sample_submission_M0L0uXE.csv'

In [3]:
oof_files = os.listdir(OOF_PATH)
removed = [i for i in oof_files if "stack" in i]
removed = removed+[i for i in oof_files if "Stack" in i]

oof_files = [i for i in oof_files if i not in removed]

oof_dict = {}
for file in oof_files:
    print(file)
    _ = pd.read_csv(f'{OOF_PATH}/{file}')
    _.columns[0]
    val = (float(_.columns[0][-5:]))
    oof_dict[file] = val

catboost_10fld_20180916082333.csv
catboost_10fld_bagtemp_20180916173347.csv
catboost_10fld_depth10_l2_1.1_temp0.1_20180916111008.csv
catboost_10fld_depth10_l2_1.1_temp1.2_20180916094559.csv
catboost_10fld_depth10_temp0.2_20180916111110.csv
catboost_10fld_depth10_temp0.3_20180916111032.csv
catboost_10fld_depth10_temp0.9_20180916094745.csv
catboost_10fld_depth10_temp1.1_20180916094605.csv
catboost_10fld_depth11_l2_1.1_temp2_20180916095129.csv
catboost_10fld_depth12_l2_1.75_temp0.5_20180916120826.csv
catboost_10fld_depth8_l2_1.1_temp0.15_20180916115751.csv
catboost_10fld_depth8_l2_1.1_temp0.5_20180916124013.csv
catboost_10fld_depth8_l2_1.1_temp3_20180916133324.csv
catboost_10fld_depth8_temp0.2_20180916120941.csv
catboost_10fld_depth8_temp0.3_20180916120820.csv
catboost_10fld_depth9_l2_1.1_temp0.55_20180916120924.csv
catboost_10fld_depth9__seed_temp0.2_20180916115740.csv
catboost_10fld_depth_10_temp0.9_20180916124610.csv
catboost_10fld_depth_10_temp5.0_20180916133913.csv
catboost_10fld_dep

In [4]:
#sorted(oof_dict.iteritems(), key=lambda (k,v): (v,k))
import operator
sorted_x = sorted(oof_dict.items(), key=operator.itemgetter(1))

In [5]:
ix = [(i[1])>0.527 for i in sorted_x]

new_dict = {}
for i in range(len(sorted_x)):
    if ix[i]:
        k,v = sorted_x[i]
        new_dict[k] = v

In [6]:
new_dict

{'catboost_5fld_baggingtemp0.85_20180916070735.csv': 0.5275,
 'catboost_10fld_bagtemp_20180916173347.csv': 0.5282,
 'catboost_10fld_20180916082333.csv': 0.531}

In [7]:
# oof_files = ['catboost_10fld_bagtemp_20180916173347.csv','catboost_10fld_20180916082333.csv',
#              'catboost_5fld_baggingtemp0.85_20180916070735.csv',
#             'xgb_10fld_20180915194445.csv','xgb_10fld_20180915183532.csv',
#              'lightgbm_10fld_20180915124747.csv',
#              'logreg_20180915130548.csv'
#             ]
oof_files = [i for i,j in new_dict.items()]
oof_files
test_files = oof_files
Y_true = pd.read_csv(f'{PATH}/{train_csv}')['is_promoted']
submission = pd.read_csv(f'{PATH}/{submit_csv}')

In [8]:
oof_df = pd.concat([pd.read_csv(f'{OOF_PATH}/{i}') for i in oof_files],axis=1)
test_df = pd.concat([pd.read_csv(f'{submission_path}/{i}').iloc[:,1:] for i in oof_files],axis=1)

In [9]:
Y_true.shape[0] == oof_df.shape[0]

True

In [10]:
Y_true.shape

(54808,)

In [11]:
test_df.shape

(23490, 3)

In [12]:
oof_df.shape

(54808, 3)

In [13]:
oof_df.head()

,catboost_5fld_baggingtemp0.85_0.5275,catboost_10fld_bagtemp_0.5282,catboost_10fld_0.531
0,0.197621,0.229450,0.233136
1,0.019009,0.015912,0.020193
2,0.000200,0.000013,0.000096
3,0.000035,0.000012,0.000031
4,0.001812,0.005401,0.002351


In [14]:
test_df.columns = oof_df.columns
test_df.head()

,catboost_5fld_baggingtemp0.85_0.5275,catboost_10fld_bagtemp_0.5282,catboost_10fld_0.531
0,0.139478,0.126459,0.137725
1,0.000424,0.000322,0.000372
2,0.000056,0.000022,0.000029
3,0.001002,0.000764,0.000878
4,0.000984,0.000743,0.000848


In [15]:
def kfold_xgb(train_df,test_df, target,num_folds, stratified = False, debug= False,modelname="lightgbm_0"):
    # Divide in training/validation and test data
    train_df = train_df
    test_df = test_df
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['employee_id','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], target)):
        train_x, train_y = train_df[feats].iloc[train_idx], target[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], target[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = XGBClassifier(
            nthread=4,
            n_estimators=5000,
            objective='binary:logistic',
            learning_rate=0.005,
#            num_leaves=34,
#            colsample_bytree=0.75,
#            subsample=0.9,
#             max_depth=16,
#             reg_alpha=0.35,
#             reg_lambda=2,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)
        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
#         oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
#         sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        print('Fold %2d F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.3).astype(int))))
        print('Fold %2d > 0.2 F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.2).astype(int))))
        print('Fold %2d > 0.1 F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.1).astype(int))))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full > 0.28 f1 score %.6f' % f1_score(target, (oof_preds>0.28).astype(int)))
    print('Full > 0.30 f1 score %.6f' % f1_score(target, (oof_preds>0.29).astype(int)))
    print('Full > 0.32 f1 score %.6f' % f1_score(target, (oof_preds>0.30).astype(int)))
    print('Full > 0.25 f1 score %.6f' % f1_score(target, (oof_preds>0.31).astype(int)))
    print('Full > 0.275 f1 score %.6f' % f1_score(target, (oof_preds>0.32).astype(int)))
    
    # Write submission file and plot feature importance
    if not debug:
        _ = datetime.now().strftime('%Y%m%d%H%M%S')
        Fname = 'F:/AV/WNS/submission/stack/'+str(modelname)+'_'+str(_)+'.csv'
        submission['is_promoted'] = sub_preds
        submission[['employee_id', 'is_promoted']].to_csv(Fname, index= False)
        oof = pd.DataFrame(oof_preds)
        score = f1_score(target, (oof_preds>0.3).astype(int))
        oof.columns = [modelname+'_'+str(round(score,4))]
        OOF_Fname = 'F:/AV/WNS/oof/stack/'+str(modelname)+'_'+str(_)+'.csv'
        oof.to_csv(OOF_Fname,index=False)
    #display_importances(feature_importance_df)
    return 

In [16]:
import gc
oof = kfold_xgb(oof_df,test_df, Y_true,num_folds=4, stratified = True, debug= False,modelname="XGB_Stack_Final_v2")

Starting LightGBM. Train shape: (54808, 3), test shape: (23490, 3)
[0]	validation_0-auc:0.903147	validation_1-auc:0.899743
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.912263	validation_1-auc:0.908174
[200]	validation_0-auc:0.914913	validation_1-auc:0.910934
[300]	validation_0-auc:0.915307	validation_1-auc:0.911146
[400]	validation_0-auc:0.915431	validation_1-auc:0.911408
[500]	validation_0-auc:0.915416	validation_1-auc:0.91129
Stopping. Best iteration:
[375]	validation_0-auc:0.91541	validation_1-auc:0.911451

Fold  1 F-score : 0.529153
Fold  1 > 0.2 F-score : 0.482782
Fold  1 > 0.1 F-score : 0.367839
[0]	validation_0-auc:0.903288	validation_1-auc:0.899906
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.91163	validati

In [17]:
#0.525448